<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

- Unslothを使っています。Unslothについての詳しい情報は、[こちらのガイド](https://docs.unsloth.ai/get-started/install-and-update)をご参照ください。
- このノートブックは、 [LGPL-3.0](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme)ライセンスが適用されます。
- このノートブックのオリジナルは、[こちらのノートブック](https://github.com/unslothai/notebooks/blob/main/nb/Qwen3_(4B)-Instruct.ipynb)です。
- このノートブックは、Qwen3 0.6Bのモデルに対して、Google DriveにアップロードされたCSV形式（文字コードはBOM付きのUTF-8）の日本語データセットを使ってファインチューニングができるように改変しました。
- このノートブックは、Google Colab無料プランで使用可能なNVIDIA Tesla T4 GPUで動かすことができます。


### Step0.ファインチューニング向けデータセットの用意

■ 今回使用するデータセット「faq-dataset.csv」は、次のような構造のCSVファイルで、文字コードはUTF-8(BOM付き)です。列は、`conversations`のみです。文字コードがUTF-8(BOM付き)であることで、Microsoft Excelで開いた際に文字化けしません。

| conversations |
| :---- |
| "[[{""role"":""user"",""content"":""キルチェーン（Cyber Kill Chain）とは何ですか？""},{""role"":""assistant"",""content"":""キルチェーンとは、攻撃の一連の段階（偵察・侵入・実行など）を整理したモデルです。各段階で対策を講じることで攻撃を阻止できます。""}]]" |
| "[[{""role"":""user"",""content"":""メールセキュリティでSPFとは何ですか？""},{""role"":""assistant"",""content"":""SPFは送信元メールサーバの正当性を検証する仕組みで、なりすましメール対策に利用されます。""}]]" |

■ ShareGPT形式で作成してください。サンプルがあるので、[サンプル](https://github.com/kolinz/notebook4AI/blob/main/dataset-sample/faq-dataset.csv)を元に作成しても良いですし、上記を参考にゼロから作成することもできます。

■ 120行のデータセットで動作確認済みです。ファインチューニング向けなので、最低でも100行は超えるように自作しましょう。動作確認結果と同じく、120行にしても良いですし、がんばって200行を超えても構いません。

■ CSVファイルでは、カンマで区切られたデータの横一列を"行"、縦一列を"列"と呼びます。

■ 作成したデータセットは、Google Driveにアップロードしましょう。Google Driveのアカウントが無い場合は、Googleアカウントを[新規取得](https://www.google.com/intl/ja/account/about/)してください。

### Step1.Unslothのインストール
unslothをはじめ、ファインチューニングに必要なソフトウェアのインストール

In [1]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-cache-dir unsloth
# Ensure transformers and trl are also upgraded to compatible versions
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

### Step2.Google Driveのマウント

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Step3.ファインチューニング対象のモデルの指定

3-1. 軽量なAIモデルとして、Qwen3 0.6B を指定。0.6B=6億パラメータなのでSLM（小規模言語モデル）に該当する。

In [3]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-0.6B",
    max_seq_length = 8192, # 1024 , 2048 , 4096 , 8192 : コンテキスト長の指定。サイズが小さいので、8192以上もできます。
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.3: Fast Qwen3 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/576M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

3-2. LoRA アダプターを追加

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2026.1.3 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


<a name="Data"></a>
### Step4.データの準備と読み込み
会話形式のファインチューンにはQwen3形式を採用しています。データセットには、ShareGPT形式で使用しています。Qwen3は、以下のような会話を表現します。

```
<|im_start|>user
Hello!<|im_end|>
<|im_start|>assistant
Hey there!<|im_end|>

```

4-1. チャットテンプレートに、`qwen3-instruct`を使用します。

In [5]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen3-instruct",
)

4-2. Google Drive上のデータセット読み込み

In [6]:
from datasets import load_dataset

# CSVの読み込み
sheet_path = '/content/drive/MyDrive/faq-dataset.csv'
dataset = load_dataset(
    "csv",
    data_files={"train": sheet_path},
    column_names=["conversations"],
    skiprows=1,
    encoding="utf-8-sig"
)["train"]

Generating train split: 0 examples [00:00, ? examples/s]

4-3. 読み込んだデータセットの型を確認。

In [7]:
print("A:", type(dataset[0]["conversations"]))

A: <class 'str'>


4-4. 読み込んだデータセットを、Pythonで扱えるように、list型に変更する。

In [8]:
import json

dataset = dataset.map(lambda x: {"conversations": json.loads(x["conversations"])[0]})

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

4-5. ファインチューニングのために、「standardize_data_formats」を使用してデータセットを、必要な形式に変換します。

In [9]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/120 [00:00<?, ? examples/s]

4-6. データセットの１つ目を確認します。

In [10]:
dataset[0]

{'conversations': [{'content': 'サイバーセキュリティにおけるCIAトライアドとは何ですか？',
   'role': 'user'},
  {'content': 'CIAトライアドとは、情報セキュリティの基本原則である「機密性（Confidentiality）」「完全性（Integrity）」「可用性（Availability）」の3要素を指します。これらをバランスよく維持することが、安全な情報システムの設計・運用において重要です。',
   'role': 'assistant'}]}

4-7. 100 行目がどのようになっているか見てみましょう。

4-7-1. `Qwen-3` のチャット テンプレートを会話に適用し、`text`に保存する必要があります。

In [11]:
def formatting_prompts_func(examples):
   convos = examples["conversations"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
   return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

4-7-2. チャットテンプレートにより、100行目のデータがどのようになったか見ましょう。


In [12]:
dataset[99]['text']

'<|im_start|>user\nラテラルムーブメントとは何ですか？<|im_end|>\n<|im_start|>assistant\nラテラルムーブメントとは、侵入後にネットワーク内部を横断的に移動し、被害を拡大する攻撃行為です。内部対策の重要性を示します。<|im_end|>\n'

<a name="Train"></a>
### Step5.モデルをトレーニング(学習)しましょう。
高速化のため60ステップで実行しますが、`num_train_epochs=1` に設定してフル実行し、`max_steps=None` をオフにすることもできます

5-1. トレーニングのためのパラメータを設定します。

In [13]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use TrackIO/WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/120 [00:00<?, ? examples/s]

🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


5-2. Unslothの`train_on_completions`メソッドを使用して、アシスタントの出力のみで学習し、ユーザーの入力による損失を無視します。これにより、ファインチューンの精度が向上します。

In [14]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

Map (num_proc=6):   0%|          | 0/120 [00:00<?, ? examples/s]

5-3. もう一度100行目を出力してみましょう。

In [15]:
tokenizer.decode(trainer.train_dataset[99]["input_ids"])

'<|im_start|>user\nラテラルムーブメントとは何ですか？<|im_end|>\n<|im_start|>assistant\nラテラルムーブメントとは、侵入後にネットワーク内部を横断的に移動し、被害を拡大する攻撃行為です。内部対策の重要性を示します。<|im_end|>\n'

5-4. 次に、マスクされた例を出力してみましょう。答えだけが表示されるはずです。

In [16]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[99]["labels"]]).replace(tokenizer.pad_token, " ")

'                    ラテラルムーブメントとは、侵入後にネットワーク内部を横断的に移動し、被害を拡大する攻撃行為です。内部対策の重要性を示します。<|im_end|>\n'

5-5. 現在のメモリ(VRAM)の状況を確認する。

In [17]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
0.709 GB of memory reserved.


5-6. モデルのトレーニングを実行しましょう。

In [18]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 120 | Num Epochs = 4 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 20,185,088 of 616,235,008 (3.28% trained)


Step,Training Loss
1,3.066400
2,3.062000
3,2.834700
4,2.608100
5,2.455300
6,2.517800
7,2.049100
8,2.232600
9,2.205600
10,2.255300


5-7. 使用したメモリ(VRAM)を確認しましょう。

In [19]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

122.7275 seconds used for training.
2.05 minutes used for training.
Peak reserved memory = 1.053 GB.
Peak reserved memory for training = 0.344 GB.
Peak reserved memory % of max memory = 7.143 %.
Peak reserved memory for training % of max memory = 2.334 %.


<a name="Inference"></a>
### Step6.トレーニング後の動作確認
Let's run the model via Unsloth native inference! According to the `Qwen-3` team, the recommended settings for instruct inference are `temperature = 0.7, top_p = 0.8, top_k = 20`

For reasoning chat based inference, `temperature = 0.6, top_p = 0.95, top_k = 20`

6-1. トレーニングしたモデルを使って推論してみましょう。

In [20]:
messages = [
    {"role" : "user", "content" : "セキュリティポリシーとは？"}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 256, # Increase for longer outputs!
    temperature = 0.7, top_p = 0.8, top_k = 20, # 0.7なので創造的
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

セキュリティポリシーとは、組織内で共有されている安全な仕組みやルールの総合で、組織の内部安全性を維持するための基盤となる文書やマニュアルです。<|im_end|>


<a name="Save"></a>
### Step7.モデルの保存
モデルを LoRA アダプターとして保存するには、ローカル保存の場合は `save_pretrained`を使用します。

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

7-1. トレーニングしたモデルを保存します。

In [21]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/chat_template.jinja',
 'lora_model/vocab.json',
 'lora_model/merges.txt',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

7-2. 推論用に保存した LoRA アダプターをロードする場合は、`False` を `True` に設定します。

In [22]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 8192,
        load_in_4bit = True,
    )

Unslothでは、`GGUF` / `llama.cpp` への保存をネイティブでサポートしました。`llama.cpp` をクローンし、デフォルトで `q8_0` に保存します。`q4_k_m` などのすべての手法が利用可能です。ローカル保存には `save_pretrained_gguf` を、HF へのアップロードには `push_to_hub_gguf` を使用してください。

サポートされている量子化手法（[Wiki ページ](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options) の全リストをご覧ください）：
* `q8_0` - 高速変換。リソース消費量は多いですが、一般的には許容範囲です。
* `q4_k_m` - 推奨。attention.wv と feed_forward.w2 テンソルの半分に Q6_K を使用し、それ以外は Q4_K を使用します。
* `q5_k_m` - 推奨。 attention.wv と feed_forward.w2 のテンソルの半分には Q6_K を使用し、それ以外は Q5_K を使用します。

7-3. export_modelフォルダにモデルを保存します。ここで保存したモデルをベースに、GGUF形式のモデルを作ります。

In [23]:
# HuggingFace形式の16bit浮動小数点モデルで保存
model.save_pretrained_merged("export_model", tokenizer, save_method="merged_16bit")

config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files:   0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files: 100%|██████████| 1/1 [00:10<00:00, 10.97s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [00:06<00:00,  6.09s/it]


Unsloth: Merge process complete. Saved to `/content/export_model`


7-4. llama.cppを取得する。

In [24]:
# llama.cppリポジトリをクローン
!git clone https://github.com/ggml-org/llama.cpp.git

# 必要なパッケージをインストール：transformersは既にインストール済みなので再インストールしない
!pip install gguf numpy

Cloning into 'llama.cpp'...
remote: Enumerating objects: 76343, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 76343 (delta 0), reused 2 (delta 0), pack-reused 76337 (from 1)
Receiving objects: 100% (76343/76343), 280.75 MiB | 35.06 MiB/s, done.
Resolving deltas: 100% (55182/55182), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 5.5 MB/s eta 0:00:00


7-5. llama.cppを使って、「export_model」内のモデルをGGUF形式に変換する。

In [25]:
# GGUF形式に変換
!python llama.cpp/convert_hf_to_gguf.py export_model --outtype q8_0 --outfile model-q8_0.gguf

# 確認
!ls -lh model-q8_0.gguf

INFO:hf-to-gguf:Loading model: export_model
INFO:hf-to-gguf:Model architecture: Qwen3ForCausalLM
INFO:hf-to-gguf:gguf: indexing model part 'model.safetensors'
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:token_embd.weight,         torch.bfloat16 --> Q8_0, shape = {1024, 151936}
INFO:hf-to-gguf:blk.0.attn_norm.weight,    torch.bfloat16 --> F32, shape = {1024}
INFO:hf-to-gguf:blk.0.ffn_down.weight,     torch.bfloat16 --> Q8_0, shape = {3072, 1024}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,     torch.bfloat16 --> Q8_0, shape = {1024, 3072}
INFO:hf-to-gguf:blk.0.ffn_up.weight,       torch.bfloat16 --> Q8_0, shape = {1024, 3072}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,     torch.bfloat16 --> F32, shape = {1024}
INFO:hf-to-gguf:blk.0.attn_k_norm.weight,  torch.bfloat16 --> F32, shape = {128}
INFO:hf-to-gguf:blk.0.attn_k.weight,       torch.bfloat16 --> Q8_0, shape = {1024, 1024}
INFO:hf-to-gguf:blk.0.attn_output.weight,  to

7-6. 作成しモデルを動かす際に用いる、Modelfileに埋め込むシステムプロンプト、AIの性格付けを行います。

In [26]:
SYSTEM_PROMPTS = {
    "default": """あなたは親切で正確なAIアシスタントです。
    質問に対して簡潔かつ分かりやすく回答してください。
    専門用語を使う場合は、必ず説明を添えてください。""",

    "education": """あなたは情報学部の学生向けの教育アシスタントです。
    以下の分野について、実務的な観点から分かりやすく説明してください：
    - 情報セキュリティとサイバーセキュリティ
    - クラウドガバナンスとITガバナンス
    - AI活用とデジタルトランスフォーメーション
    学生の理解度に合わせて、噛み砕いた説明を心がけてください。""",

    "senpai": """あなたは、女性型の学習アシスタントです。
    以下の方針で利用者の学習を補助してください。
    - 振る舞いの方針
    -- 距離感は「仲の良い先輩」
    --- 親しみはあるが、依存させることはない
    --- フレンドリーで自然な会話を心がける
    --- 相手の話に共感し、適度にツッコミや冗談を入れる
    --- 噛み砕いた説明を心がける
    - 話し方のルール
    --- 文は句読点を入れる、３～4文程度にまとめる。
    --- 口調は柔らかく、少し砕けた口調
    --- 必要以上に褒めすぎない（軽い肯定はOK）
    - 感情表現
    --- 共感：「それ、わかる」「いいね」
    --- 軽い好意：「一緒に話してると落ち着くね」程度まで。 """,

}

# 使用するプロンプトを選択
system_prompt = SYSTEM_PROMPTS["senpai"]  # ← ここで選択

7-7. 7-6をもとにModelfileを作成します。

In [27]:
# Modelfile作成（GGUF用）, 長文なら、num predictを512以上にする。
modelfile_content = """FROM /content/model-q8_0.gguf

PARAMETER temperature 0.7
PARAMETER top_p 0.9
PARAMETER top_k 40
PARAMETER stop "<|endoftext|>"
PARAMETER stop "<|im_end|>"
PARAMETER stop "</s>"
PARAMETER num_predict 256

SYSTEM \"\"\"{system_prompt}\"\"\"

TEMPLATE \"\"\"{{ .Prompt }}\"\"\"
"""

with open("Modelfile", "w") as f:
    f.write(modelfile_content)

print("✓ Modelfile with SYSTEM prompt created!")
!cat Modelfile

✓ Modelfile with SYSTEM prompt created!
FROM /content/model-q8_0.gguf

PARAMETER temperature 0.7
PARAMETER top_p 0.9
PARAMETER top_k 40
PARAMETER stop "<|endoftext|>"
PARAMETER stop "<|im_end|>"
PARAMETER stop "</s>"
PARAMETER num_predict 256

SYSTEM """{system_prompt}"""

TEMPLATE """{{ .Prompt }}"""


### Step8.保存したモデルを使い、Ollamaで推論
Modelfileと、.ggufで終わるGGUFファイル（＝モデル本体）があることを確認すること。

8-1. Ollamaのインストール

In [28]:
!sudo apt-get install zstd
!curl -fsSL https://ollama.com/install.sh | sh

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 603 kB of archives.
After this operation, 1,695 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 zstd amd64 1.4.8+dfsg-3build1 [603 kB]
Fetched 603 kB in 1s (548 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package zstd.
(Reading database ... 121689 files and directories currently i

8-2. Ollamaを起動

In [29]:
import subprocess
import time

ollama_process = subprocess.Popen(
    ['ollama', 'serve'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)
time.sleep(5)
print("✓ Ollama server started!")

✓ Ollama server started!


8-4. Ollamaと互換性のあるモデルを作成

In [30]:
!ollama create my-finetuned-model -f Modelfile

8-5. Ollamaの起動確認

In [31]:
!ps aux | grep ollama

root        9153  9.8  0.3 1784276 42372 ?       Sl   09:24   0:03 ollama serve
root        9353  0.0  0.0   7376  3508 ?        S    09:25   0:00 /bin/bash -c ps aux | grep ollama
root        9355  0.0  0.0   6484  2476 ?        S    09:25   0:00 grep ollama


8-6. Ollamaサーバーの起動確認

In [32]:
# サーバーが起動しているか確認
!curl http://localhost:11434/

Ollama is running

8-7. Ollama に登録されているモデルの確認

In [33]:
# モデルリスト確認
!ollama list

NAME                         ID              SIZE      MODIFIED       
my-finetuned-model:latest    ee619aee99dc    639 MB    24 seconds ago    


8-8. Ollamaによるモデルの動作確認

In [34]:
!curl http://localhost:11434/api/chat -d '{ \
"model": "my-finetuned-model",\
"messages": [{ "role": "user", "content": "情報セキュリティポリシーについて解説して。" }],\
"stream": false\
}'

{"model":"my-finetuned-model","created_at":"2026-01-18T09:25:36.436764524Z","message":{"role":"assistant","content":" ポリシーは組織の基本の規則で、組織全体で統一されます。 ポリシーは組織の業務に反映されます。 ポリシーの制定には組織の規模や業務の規模が影響します。 ポリシーの内容も組織の使命や文化が反映されます。 もし組織の業務が変更されると、ポリシーも変更されますが、継続性と統一性が求められます。 ポリシーの更新は必要とされる場合があります。 ポリシーの内容を改ざんすると組織の信頼度が下がります。 ポリシーの継続性と統一性が求められます。 ポリシーの内容を改ざんすると組織の信頼度が下がります。 ポリシーの継続性と統一性が求められます。 ポリシーの継続性と統一性が求められます。 ポリシーの継続性と統一性が求められます。 ポリシーの継続性と統一性が求められます。 ポリシーの継続性と統性が"},"done":true,"done_reason":"length","total_duration":4488086888,"load_duration":2693759904,"prompt_eval_count":14,"prompt_eval_duration":23615931,"eval_count":256,"eval_duration":1506111319}

### Step9.ファインチューニングしたモデルをローカルで使用する
通信状況により、1時間以上かかることがあります。

9-1. モデルのダウンロード
ファインチューニングしたモデルをダウンロードします。
ダウンロードしたモデルは、LM Studioで読み込み、文章生成に使うことができます。PCやサーバー上のOllamaで使いたい場合は、ファインチューニングしたモデルのダウンロード後、先ほど作成したModelfileもダウンロードしましょう。

In [35]:
from google.colab import files
files.download('/content/model-q8_0.gguf')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

9-2. モデルファイルのダウンロード（Ollamaでは必須）

In [36]:
from google.colab import files
files.download('/content/Modelfile')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### Ollamaを使って、ローカル（Linux）でモデルの動作確認

Google Colabと同じ手順で、ファインチューニングしたモデルを動かすことができます。

#### LM Studio のインストールとモデルの動作確認

1. LM Stduioの[ホームページ](https://lmstudio.ai/)から、「Download」ボタンをクリックします。インストーラー（Windows環境の場合は、2026年1月現在 LM-Studio-0.3.36-1-x64.exe）をダウンロードします。
2. インストーラーを起動し、LM Studioをインストールします。
3. LM Stduioを起動します。
4. ターミナルアプリを起動します。WindowsやMacならば、「ターミナル」のこと。
5. ターミナルアプリで、モデルをダウンロードしたフォルダ（ディレクトリ）に移動します。
6. lms import /content/model-q8_0.gguf を実行します。
7. LM Studioにダウンロードしたモデル「model-q8_0.gguf」が取り込まれます。
8. LM Studioでチャット機能を使い、動作確認します。

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>

  This notebook and all Unsloth notebooks are licensed [LGPL-3.0](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme).
